In [1]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor



In [2]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')

In [3]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [61]:
import functions
import transformers
import processors
import importlib
import builders

importlib.reload(functions)
importlib.reload(transformers)
importlib.reload(processors)
importlib.reload(builders)


from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer


from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor

from processors.basic_pre_processor import BasicPreProcessor

from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder


importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)


importlib.reload(processors.basic_pre_processor)

importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)









<module 'builders.reg_pipe_builder' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\builders\\reg_pipe_builder.py'>

In [5]:
names = ['BetterBot', 'STEEBot', 'HastyBot']
bots = BotExtarctor(names)(train)
is_player = ~IsBotExtarctor(names)(train)


In [6]:
test = train
transformer = SelectTransformer(is_player)
transformer.transform(test)


,nickname,score,rating
game_id,,,
1,stevy,429,1500
3,davidavid,440,1811
4,Inandoutworker,119,1473
5,stevy,325,1500
6,HivinD,378,2029
...,...,...,...
72767,HAYDEN,340,1395
72770,samsiah06,97,1332
72771,BB-8,390,1500


In [7]:
train_pipe = (Pipeline([('bot_names', ColumnsSetterTransformer({'bot_name': BotExtarctor(names),})),
                   ('players_selection', SelectTransformer(is_player)),
                   ('set_bot_training', ColumnsSetterTransformer({'bot_rating': train[~is_player]['rating']})),
                   ('name_dropper', NameDropperTransformer(['nickname']))
                   ]))

games_pipe = (Pipeline([('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                        ('drop_first', NameDropperTransformer('first'))
                       ]))

new_train = train_pipe.transform(train)
new_games = games_pipe.transform(games).merge(new_train, left_index=True, right_index=True)

In [8]:
# new_games = games.merge(new_train, left_index=True, right_index=True)
# new_games

In [9]:
# is_player = ~IsBotExtarctor(names)(train)
# bots = BotExtarctor(names)(train)
# train_no_bots = train[is_player]
# train_no_bots['bot_name'] = bots
# train_no_bots['bot_rating'] = train[~is_player]['rating']
# train_no_bots = train_no_bots.drop(columns=['nickname'])
# games_with_rating = games.merge(train_no_bots, left_index=True, right_index=True)
# train_no_bots


In [10]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [11]:
trans = SelectRowsTransformer(train.index)
train_turns = trans.transform(turns)

In [12]:
names = ['BetterBot', 'STEEBot', 'HastyBot']

turns_pipe = Pipeline([('is_player_extractor', ColumnsSetterTransformer({'is_player_turn': IsBotExtarctor(names, neg=True)})),
                       ('word_info_mappers', MapSetTransformer({'move_len': (lambda x: len(x) if type(x) == str else np.NAN, 'move'),
                                                                'reused_num': (lambda x: x.count('.') if type(x) == str else np.NAN, 'move'),
                                                                'jokers_num': (lambda x: sum(1 for c in x if c.islower()) if (type(x) == str) and (x not in ['(challenge)', '(time)']) else np.NAN, 'move')
                                                                })),
                       ('to_series', SeriesFromGroupTransformer('game_id', 'turn_number'))
                       ])

new_turns = turns_pipe.transform(turns)
new_turns

1                      nickname     rack location      ...
2                      nickname     rack location      ...
3                      nickname     rack location      ...
4                           nickname     rack location ...
5                    nickname     rack location       m...
                               ...                        
72769                 nickname     rack location      m...
72770                  nickname     rack location      ...
72771                 nickname     rack location     mo...
72772                  nickname     rack location      ...
72773                 nickname     rack location     mo...
Length: 72773, dtype: object

In [13]:
more_games_pipe = Pipeline([('assign_turns', ColumnsSetterTransformer({'turns': new_turns}))])

new_games = more_games_pipe.transform(new_games)
new_games.head()

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating,turns
game_id,,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637,nickname rack location ...
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071,nickname rack location ...
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936,nickname rack location ...
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844,nickname rack location m...
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143,nickname rack location mo...


In [14]:
new_games['turns'][4]

,nickname,rack,location,move,points,score,turn_type,is_player_turn,move_len,reused_num,jokers_num
turn_number,,,,,,,,,,,
1,BetterBot,ADEMSTX,8D,MAXED,36,36,Play,False,5.0,0.0,0.0
2,Inandoutworker,DISTVWY,9E,WIT,24,24,Play,True,3.0,0.0,0.0
3,BetterBot,DEJRSTU,7G,JET,40,76,Play,False,3.0,0.0,0.0
4,Inandoutworker,ADLOSVY,J7,SLAY,26,50,Play,True,4.0,0.0,0.0
5,BetterBot,DERRRSU,E4,REDR..,20,96,Play,False,6.0,2.0,0.0
6,Inandoutworker,BBDGOUV,NaN,-BBGOUV,0,50,Exchange,True,7.0,0.0,0.0
7,BetterBot,EFIMRSU,D1,FUME,30,126,Play,False,4.0,0.0,0.0
8,Inandoutworker,DEEEGOR,1A,REE.,21,71,Play,True,4.0,1.0,0.0
9,BetterBot,EINNORS,E4,......N,11,137,Play,False,7.0,6.0,0.0


In [15]:
# moments_pipe = Pipeline([('points_moments', MapSetTransformer({'p_points_1_moment': (lambda turns: (turns[turns['is_player_turn']]['points']).mean(), 'turns'),
#                                                             'p_points_std': (lambda turns: turns[turns['is_player_turn']]['points'].std(), 'turns'),
#                                                             'b_points_1_moment': (lambda turns: (turns[~turns['is_player_turn']]['points']).mean(), 'turns'),
#                                                             'b_points_std': (lambda turns: turns[~turns['is_player_turn']]['points'].std(), 'turns'),
#                                                             'a_points_1_moment': (lambda turns: (turns['points']).mean(), 'turns'),
#                                                             'a_points_std': (lambda turns: turns['points'].std(), 'turns'),
#                                                             },
# ))])
# res = moments_pipe.transform(new_games)


              

In [16]:
# res.head()

In [17]:
# res['turns'][61248]

In [18]:
turns[turns['move'].isna()]

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
377,27,NurdiniDelisha,ADFNTTU,O8,NaN,16,249,NaN
409,22,akiff_faleel,AANU,14J,NaN,12,350,NaN
420,23,Rick,AGINU,M12,NaN,19,409,NaN
536,13,HastyBot,AAEILNN,F5,NaN,14,316,NaN
630,2,220674_ZAVEPHUA,AAEINNR,7H,NaN,11,11,NaN
...,...,...,...,...,...,...,...,...
72252,10,akiff_faleel,CGLLNUX,D10,NaN,12,238,Play
72395,18,gppa,AEINRSU,A14,NaN,35,272,NaN
72467,7,Saryk,?AALNOU,9F,NaN,11,74,NaN


In [19]:
pd.get_dummies(games['lexicon'], prefix='lexicon', dummy_na=True).sum()


lexicon_CSW21     44594
lexicon_ECWL       6925
lexicon_NSWL20        4
lexicon_NWL20     21250
lexicon_nan           0
dtype: int64

In [20]:
trans = OneHotEncoderTransformer(['lexicon', 'rating_mode'])
trans.transform(games).head()

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,lexicon_nan,rating_mode_CASUAL,rating_mode_RATED,rating_mode_nan
game_id,,,,,,,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,0,0,0,1,0,1,0,0
2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418,1,0,0,0,0,0,1,0
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,1,0,0,0,0,0,1,0
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,1,0,0,0,0,1,0,0
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,0,0,0,1,0,1,0,0


In [21]:
games.isna().sum()

first                    0
time_control_name        0
game_end_reason          0
winner                   0
created_at               0
lexicon                  0
initial_time_seconds     0
increment_seconds        0
rating_mode              0
max_overtime_minutes     0
game_duration_seconds    0
dtype: int64

In [22]:
# need to start cv and gridsearch

In [23]:
new_games.head()

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating,turns
game_id,,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637,nickname rack location ...
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071,nickname rack location ...
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936,nickname rack location ...
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844,nickname rack location m...
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143,nickname rack location mo...


In [8]:
from sklearn.ensemble import RandomForestRegressor
y = new_games['rating']
hot_drop_pipe = Pipeline([('hot', OneHotEncoderTransformer(['time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),
                          ('add_turns', AddToDictTransformer({'turns': turns})),
                        ('drops', NameDropperTransformer(['created_at', 'rating', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name', 'turns'], target='games')),
                        ('get_games', GetFromDictTransformer('games')),
                        ('tree', RandomForestRegressor(max_depth=2, random_state=0)),
                        ])

hot_drop_pipe.fit(new_games, y)


NameError: name 'new_games' is not defined

In [61]:
from sklearn.metrics import mean_squared_error
y_pred = hot_drop_pipe.predict(new_games)
np.sqrt(mean_squared_error(y, y_pred))

160.6309598124796

In [25]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)

RandomForestRegressor(max_depth=2, random_state=0)

In [26]:
from sklearn.metrics import mean_squared_error
y_pred = regr.predict(x)
np.sqrt(mean_squared_error(y, y_pred))

160.6309598124796

In [62]:
cv_res = cross_validate(hot_drop_pipe, new_games, y, cv=3,
                         scoring=('r2', 'neg_root_mean_squared_error'),
                         return_train_score=True)

In [63]:
cv_res

{'fit_time': array([2.6080668 , 2.49774575, 3.07307911]),
 'score_time': array([0.0984869 , 0.07749319, 0.07101583]),
 'test_r2': array([0.51327613, 0.5102373 , 0.50894537]),
 'train_r2': array([0.51280524, 0.51703496, 0.50914726]),
 'test_neg_root_mean_squared_error': array([-160.32339965, -160.27940283, -161.27550924]),
 'train_neg_root_mean_squared_error': array([-160.25010724, -159.82309691, -160.73137027])}

In [27]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [28]:
dict = {'games': games, 'turns': turns, 'train': train}

In [119]:
names = ['BetterBot', 'STEEBot', 'HastyBot']

testpipe = Pipeline([('test_turns_remove', SelectRowsTransformer(train.index, target='turns')),
                     ('test_games_remove',  SelectRowsTransformer(train.index.unique(), target='games')),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src='train', dest='train')),
                    ('turns_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src='turns', dest='turns')),  
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src='train', dest='games')),
                     ('is_player_extractor', ColumnsSetterTransformer({'is_player_turn': IsBotExtarctor(names, 'nickname', True)}, target='turns')),
                     ('turns_word_info_mappers', MapSetTransformer({'move_len': (lambda x: len(x) if type(x) == str else np.NAN, 'move'),
                                                              'reused_num': (lambda x: x.count('.') if type(x) == str else np.NAN, 'move'),
                                                              'jokers_num': (lambda x: sum(1 for c in x if c.islower()) if (type(x) == str) and (x not in ['(challenge)', '(time)']) else np.NAN, 'move')
                                                            }, target='turns')),
                     ('points_moments', ExtractSetColumnsTransformer({'p_points_1_moment': 
                                                                     (lambda turns: turns[turns['is_player']]['points'].groupby('game_id').mean())
                                                                     }, src = 'turns', dest = 'games')),
                      # ('get_games', GetFromDictTransformer('games'))
                     ])


result = testpipe.transform(dict)

In [66]:
result['turns']

,turn_number,nickname,rack,location,move,points,score,turn_type,is_player,is_player_turn,move_len,reused_num,jokers_num
game_id,,,,,,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play,False,False,3.0,0.0,0.0
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play,True,True,3.0,0.0,0.0
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play,False,False,4.0,0.0,0.0
1,4,stevy,EMORSUX,5K,UM,16,34,Play,True,True,2.0,0.0,0.0
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play,False,False,8.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play,True,True,2.0,0.0,0.0
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play,False,False,4.0,1.0,0.0
72773,24,adola,ABRRU,15E,BRA.,7,383,Play,True,True,4.0,1.0,0.0


In [173]:
names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([('get_relavent_turns', SelectRowsTransformer(train.index, target='turns')),
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target='games')),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src='train', dest='train')),
                    ('turns_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src='turns', dest='turns')),  
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src='train', dest='games')),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src='train', dest='games')),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target='train')),
                    ])
result = prePipe.transform(dict)


In [86]:
result['turns']

,turn_number,nickname,rack,location,move,points,score,turn_type,is_player
game_id,,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play,False
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play,True
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play,False
1,4,stevy,EMORSUX,5K,UM,16,34,Play,True
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play,False
...,...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play,True
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play,False
72773,24,adola,ABRRU,15E,BRA.,7,383,Play,True


In [70]:
result['games']

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,bot_rating
game_id,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,1637
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,2071
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,1936
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,1844
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,2143
...,...,...,...,...,...,...,...,...,...,...,...,...
72767,BetterBot,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,3669.988413,1952
72770,HastyBot,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,2237
72771,HastyBot,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,1614


In [115]:
result['train']

,nickname,score,rating,is_player
game_id,,,,
1,stevy,429,1500,True
3,davidavid,440,1811,True
4,Inandoutworker,119,1473,True
5,stevy,325,1500,True
6,HivinD,378,2029,True
...,...,...,...,...
72767,HAYDEN,340,1395,True
72770,samsiah06,97,1332,True
72771,BB-8,390,1500,True


In [20]:
n_games, n_turns, n_ratings = BasicPreProcessor(prePipe, games, turns).process(train, 'train')

In [135]:
n_ratings.shape

(50410,)

In [175]:
n_games.head()

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,bot_rating,bot_name
game_id,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,1637,BetterBot
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,2071,BetterBot
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,1936,BetterBot
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,1844,STEEBot
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,2143,STEEBot


In [21]:
transformers = []
to_dict = [('add_turns', AddToDictTransformer({'turns': n_turns}))]
featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')}, target='games')),
                       ('turns_word_info_mappers', MapSetTransformer({'move_len': (lambda x: len(x) if type(x) == str else np.NAN, 'move'),
                                                              'reused_num': (lambda x: x.count('.') if type(x) == str else np.NAN, 'move'),
                                                              'jokers_num': (lambda x: sum(1 for c in x if c.islower()) if (type(x) == str) and (x not in ['(challenge)', '(time)']) else np.NAN, 'move')
                                                            }, target='turns')),
                       ('points_moments', ExtractSetColumnsTransformer({'p_points_1_moment': 
                                                                     (lambda turns: turns[turns['is_player']]['points'].groupby('game_id').mean())
                                                                     }, src = 'turns', dest = 'games')),
                      ]
finalTransformers = [
                     ('hot', OneHotEncoderTransformer(['time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'], target='games'),),
                     ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'], target='games')),
                     ('get_games', GetFromDictTransformer('games')),
                        ('tree', RandomForestRegressor(max_depth=3 ,random_state=0)),
                    ]
transformers.extend(to_dict)
transformers.extend(featureTransformers)
transformers.extend(finalTransformers)
reg_pipe = Pipeline(transformers)

In [184]:
n_games.head()

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,bot_rating,bot_name
game_id,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,1637,BetterBot
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,2071,BetterBot
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,1936,BetterBot
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,1844,STEEBot
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,2143,STEEBot


In [194]:
r = reg_pipe.transform(n_games)

In [195]:
r.head()

,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,bot_rating,is_bot_first,p_points_1_moment,time_control_name_blitz,time_control_name_rapid,...,lexicon_NSWL20,lexicon_NWL20,lexicon_nan,rating_mode_CASUAL,rating_mode_RATED,rating_mode_nan,bot_name_BetterBot,bot_name_HastyBot,bot_name_STEEBot,bot_name_nan
game_id,,,,,,,,,,,,,,,,,,,,,
1,1,1200,0,1,674.844274,1637,True,30.642857,0,0,...,0,1,0,1,0,0,1,0,0,0
3,1,900,0,5,492.268262,2071,True,31.428571,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,3600,0,1,350.861141,1936,True,8.500000,0,0,...,0,0,0,1,0,0,1,0,0,0
5,0,1200,0,1,642.688722,1844,True,20.312500,0,0,...,0,1,0,1,0,0,0,0,1,0
6,0,900,0,1,426.950541,2143,False,31.500000,0,0,...,0,0,0,0,1,0,0,0,1,0


In [22]:
cv_res = cross_validate(reg_pipe, n_games, n_ratings, cv=3,
                         scoring=('r2', 'neg_root_mean_squared_error'),
                         return_train_score=True,)

In [23]:
cv_res

{'fit_time': array([11.08285809, 10.09452915, 10.93893647]),
 'score_time': array([6.36368179, 6.58686972, 7.19067025]),
 'test_r2': array([0.58056479, 0.58016383, 0.58542041]),
 'train_r2': array([0.58325108, 0.58599991, 0.58363856]),
 'test_neg_root_mean_squared_error': array([-148.82915998, -148.39688569, -148.18610384]),
 'train_neg_root_mean_squared_error': array([-148.21230608, -147.97280614, -148.03359675])}

In [66]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([('get_relavent_turns', SelectRowsTransformer(train.index, target=T_NAME)),
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                    ('turns_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=T_NAME, dest=T_NAME)),  
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')}, target=G_NAME)),
                       ('turns_word_info_mappers', MapSetTransformer({'move_len': (lambda x: len(x) if type(x) == str else np.NAN, 'move'),
                                                                      'reused_num': (lambda x: x.count('.') if type(x) == str else np.NAN, 'move'),
                                                                    # 'jokers_num': (lambda x: sum(1 for c in x if c.islower()) if (type(x) == str) and (x not in ['(challenge)', '(time)']) else np.NAN, 'move')
                                                                     }, target=T_NAME)),
                       ('points_moments', ExtractSetColumnsTransformer({'p_points_1_moment': 
                                                                      (lambda turns: turns[turns['is_player']]['points'].groupby('game_id').mean()),
                                                                        'p_points_std':
                                                                      (lambda turns: turns[turns['is_player']]['points'].groupby('game_id').std()),
                                                                        'a_poits_1_moment': 
                                                                        lambda turns: turns['points'].groupby('game_id').mean()
                                                                       }, src=T_NAME, dest=G_NAME)),
                       ('from_turns', ExtractSetColumnsTransformer({'moves_avg': 
                                                                    lambda turns: turns[turns['is_player']]['move_len'].groupby('game_id').mean(),
                                                                    'reuded_sum': 
                                                                    lambda turns: turns[turns['is_player']]['move_len'].groupby('game_id').sum()
                                                                   }, src=T_NAME, dest=G_NAME)),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        , target=G_NAME)),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'], target=G_NAME)),                                              
                      ]

preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()

In [67]:
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)
reg_pipe = RegPipeBuilder(n_games, G_NAME, n_turns, T_NAME, featureTransformers, RandomForestRegressor(random_state=0)).build()
# cv_res = cross_validate(reg_pipe, n_games, n_ratings, cv=3,
#                          scoring=('r2', 'neg_root_mean_squared_error'),
#                          return_train_score=True,)

In [68]:
reg_pipe[:-1].transform(n_games).columns

Index(['winner', 'initial_time_seconds', 'increment_seconds',
       'max_overtime_minutes', 'game_duration_seconds', 'bot_rating',
       'is_bot_first', 'p_points_1_moment', 'p_points_std', 'a_poits_1_moment',
       'moves_avg', 'reuded_sum', 'time_control_name_blitz',
       'time_control_name_rapid', 'time_control_name_regular',
       'time_control_name_ultrablitz', 'time_control_name_nan',
       'game_end_reason_CONSECUTIVE_ZEROES', 'game_end_reason_RESIGNED',
       'game_end_reason_STANDARD', 'game_end_reason_TIME',
       'game_end_reason_nan', 'lexicon_CSW21', 'lexicon_ECWL',
       'lexicon_NSWL20', 'lexicon_NWL20', 'lexicon_nan', 'rating_mode_CASUAL',
       'rating_mode_RATED', 'rating_mode_nan', 'bot_name_BetterBot',
       'bot_name_HastyBot', 'bot_name_STEEBot', 'bot_name_nan'],
      dtype='object')

In [11]:
cv_res = cross_validate(reg_pipe, n_games, n_ratings, cv=3,
                         scoring=('neg_root_mean_squared_error'),
                         return_train_score=True,)

In [12]:
cv_res

{'fit_time': array([56.48965836, 66.5057745 , 62.94231772]),
 'score_time': array([5.660676  , 5.92696142, 4.38006544]),
 'test_score': array([-107.53036969, -107.09103125, -106.96695528]),
 'train_score': array([-39.74583182, -39.80233843, -39.79982618])}

In [49]:
s = pd.concat([n_games['bot_name'], pd.Series(n_games['bot_name'].unique(), index=(-x-1 for x in range(len(n_games['bot_name'].unique()))))])

In [53]:
s = pd.concat([n_games['bot_name'], pd.Series(['BetterBot', 'HastyBot', 'STEEBot', 'h'], index=(-x-1 for x in range(4)))])

In [58]:
pd.get_dummies(s)
n_games.merge(pd.get_dummies(s), left_index=True, right_index=True)['h'].sum()

0